In [1]:
import os
import pandas as pd
import pdpipe as pdp
import spacy
import sys

from IPython.display import display
from os.path import join
from pathlib import Path

project_root = Path('..')
sys.path.append(os.path.abspath(project_root))
from notebooks.utils import init_data_dir  # noqa

from notebooks import pipes  # noqa

init_data_dir(project_root)

preprocess_path = join(project_root, Path('data/preprocess'))

nlp = spacy.load('en_core_web_sm')

df = pd.read_hdf(join(preprocess_path, Path('bawe_df.hdf5')))

# Information for the British Academic Written English Corpus

This notebook is for gathering information about the BAWE dataset. The dataset should already be parsed and stored in `data/preprocess/bawe_df.hdf5`.

In [2]:
df

,author,genre,text
0,1,a,Racism is still a problem within our society t...
1,1,b,Official statistics are those produced by eith...
2,1,c,Since the fourteenth century the practice of m...
3,1,d,Much more reproductive choice is now available...
4,2,a,Victorian notions of women's madness were larg...
...,...,...,...
2572,6998,a,E. Warwick Slinn describes dramatic monologue ...
2573,6998,b,Hugh Blair voices an attack on the practices o...
2574,6998,c,"'The first thing to remember about Donne,' wri..."
2575,6998,d,Susan Wiseman calculated that the latest possi...


In [3]:
resample_splits = True

train_df_path = join(preprocess_path, 'bawe_train_df.hdf5')
valid_df_path = join(preprocess_path, 'bawe_valid_df.hdf5')

train_df_exists = os.path.exists(train_df_path)
valid_df_exists = os.path.exists(valid_df_path)

if not (train_df_exists and valid_df_exists) or resample_splits:
    print('Resampling...')

    train_df = df.sample(frac=0.8).sort_values(by=['author', 'genre'])
    valid_df = df.drop(train_df.index)

    train_df = train_df.reset_index(drop=True)
    valid_df = valid_df.reset_index(drop=True)

    train_df.to_hdf(train_df_path, key='bawe_train_df')
    valid_df.to_hdf(valid_df_path, key='bawe_valid_df')
else:
    train_df = pd.read_hdf(train_df_path)
    valid_df = pd.read_hdf(valid_df_path)

print('Train Set:')
display(train_df)
print('Validation Set:')
display(valid_df)

Resampling...
Train Set:


,author,genre,text
0,1,a,Racism is still a problem within our society t...
1,1,b,Official statistics are those produced by eith...
2,1,c,Since the fourteenth century the practice of m...
3,1,d,Much more reproductive choice is now available...
4,2,a,Victorian notions of women's madness were larg...
...,...,...,...
2057,6215,e,<heading>INTRODUCTION</heading>Both Macrochloa...
2058,6215,f,<list></list><abstract><heading>Abstract</head...
2059,6998,c,"'The first thing to remember about Donne,' wri..."
2060,6998,d,Susan Wiseman calculated that the latest possi...


Validation Set:


,author,genre,text
0,3,d,"At the eve of the Versailles conference, promo..."
1,3,k,<heading>Introduction</heading>When we talk ab...
2,7,a,<abstract><heading>Summary</heading>The experi...
3,9,b,<heading>Introduction</heading>The study of gr...
4,10,d,The assumption that a gap existed between the ...
...,...,...,...
510,6212,d,<heading>Introduction</heading>The reaction be...
511,6215,c,<heading>INTRODUCTION</heading>Seed quality in...
512,6215,d,<heading>Question 1 Forest Growth Rates</headi...
513,6998,a,E. Warwick Slinn describes dramatic monologue ...


In [4]:
pipeline = pdp.PdPipeline([pipes.IDText(),
                           pipes.SplitText(nlp, show_loading=True)])

train_df = pipeline(train_df)
valid_df = pipeline(valid_df)

print('Train set:', flush=True)
display(train_df)
print('Validation set:', flush=True)
display(valid_df)

100%|##########| 515/515 [02:26<00:00,  4.13it/s]

Train set:


sentence
author text_id sentence_position                                                   
1      0       0                  Racism is still a problem within our society t...
               1                  Ethnic minorities are concentrated into certai...
               2                  This can be seen in a survey, carried out in 1...
               3                  In this essay I will look at what racism is, a...
               4                  In this section I will cover three areas that ...
...                                                                             ...
6998   2       175                Yerma never wants the life of a man, yet she i...
               176                She resents her ability to exercise new freedo...
               177                However, Hedda resents her inactivity and depe...
               178                She rejects motherhood and matrimony in favour...
               179                While Hedda aspires to acquire the authority o...

[238088 rows x 1 columns]

Validation set:


sentence
author text_id sentence_position                                                   
3      0       0                  At the eve of the Versailles conference, promo...
               1                  Moving with caution, Piłsudski, Head of State ...
               2                  From the outset, it appeared that Poland's ind...
               3                  It was only the realisation of Soviet hostilit...
               4                  At nearly the same time that peace was conclud...
...                                                                             ...
6998   1       70                                                   (67-69)</quote>
               71                 By allowing the child the last word, Wordswort...
               72                 Wordsworth challenges the reader's perceptions...
               73                 The poet asks if the child's cheerful percepti...
               74                 Wordsworth questions if an adult is better pre...

[58833 rows x 1 columns]

In [5]:
train_df.to_hdf(join(preprocess_path, 'bawe_train_sentences.hdf5'),
                key='bawe_train_sentences')
valid_df.to_hdf(join(preprocess_path, 'bawe_valid_sentences.hdf5'),
                key='bawe_valid_sentences')

In [6]:
237192 / 20

11859.6